In [1]:
import VariationalBayes as vb
import LogisticGLMM_lib as logit_glmm
import VariationalBayes.SparseObjectives as obj_lib
import VariationalBayes.ExponentialFamilies as ef

import matplotlib.pyplot as plt
%matplotlib inline

import autograd
import autograd.numpy as np
import scipy as sp
from scipy import sparse

import copy
from scipy import optimize

import os
import json

import time

import pickle

data_dir = os.path.join(os.environ['GIT_REPO_LOC'],
                        'LinearResponseVariationalBayes.py/Models/LogisticGLMM/data')


In [2]:
# Generate data

true_beta = np.array(range(5))
true_beta = true_beta - np.mean(true_beta)
true_mu = 0.
true_tau = 40.0

# Used 2, 5, 10, 20, 40, 60, 100
num_obs_per_group = 100
analysis_name = 'simulated_data_for_refit_{}'.format(num_obs_per_group)

true_params = logit_glmm.TrueParameters(
    num_obs_per_group = num_obs_per_group,
    num_groups = 200,
    true_beta = true_beta,
    true_mu = 0.,
    true_tau = 40.0)

x_mat, y_g_vec, y_vec  = true_params.generate_data()
prior_par = logit_glmm.get_default_prior_params(true_params.beta_dim)
glmm_par = logit_glmm.get_glmm_parameters(K=true_params.beta_dim, NG=true_params.num_groups)
logit_glmm.initialize_glmm_pars(glmm_par)
init_par_vec = glmm_par.get_free()

timer = obj_lib.Timer()

In [3]:
# Get an initial fit and preconditioner.

model = logit_glmm.LogisticGLMM(
    glmm_par=glmm_par, prior_par=prior_par, x_mat=x_mat,
    y_vec=y_vec, y_g_vec=y_g_vec, num_gh_points=5)

vb_time = time.time()
vb_opt = model.tr_optimize(init_par_vec, gtol=1e-6, maxiter=500)
opt_x = vb_opt.x

timer.tic()
kl_hess = model.get_sparse_free_hessian(opt_x, print_every_n=20)
weight_jac = model.get_sparse_weight_free_jacobian(opt_x, print_every_n=50)
moment_jac = model.moment_wrapper.get_moment_jacobian(opt_x)
timer.toc('Derivatives')


Iter  0  value:  16893.9343295
Iter  5  value:  12268.4119005
Iter  10  value:  12045.771868
Iter  15  value:  12045.763536
Optimization terminated successfully.
         Current function value: 12045.763536
         Iterations: 16
         Function evaluations: 17
         Gradient evaluations: 17
         Hessian evaluations: 0
Group 0 of 199.
Group 20 of 199.
Group 40 of 199.
Group 60 of 199.
Group 80 of 199.
Group 100 of 199.
Group 120 of 199.
Group 140 of 199.
Group 160 of 199.
Group 180 of 199.


/home/rgiordan/Documents/git_repos/autograd/autograd/core.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


Group 0 of 199
Group 50 of 199
Group 100 of 199
Group 150 of 199
Derivatives: 21.6134934425354 seconds


In [4]:
preconditioner = sp.sparse.diags([1 / np.sqrt(kl_hess.diagonal())], [0])
model.objective.preconditioner = preconditioner
cond_init = sp.sparse.linalg.spsolve(preconditioner, opt_x)

/usr/local/lib/python3.5/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:102: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


In [5]:
def simulate_and_fit(refit_model, cond_init, true_params):
    x_mat, y_g_vec, y_vec = \
        true_params.generate_data()
    refit_model.x_mat = x_mat
    refit_model.y_vec = y_vec
    refit_model.y_g_vec = y_g_vec
    #print('x mat: ', x_mat[0, 0], refit_model.x_mat[0, 0])

    vb_time = time.time()
    vb_opt = refit_model.tr_optimize_cond(
        cond_init,
        preconditioner=preconditioner,
        gtol=1e-6, maxiter=500, verbose=False)
    return vb_opt


In [6]:
opt_x_sims = []
num_sims = 200
timer.tic()
for sim in range(num_sims):
    if sim % 20 == 0:
        print('Sim {} of {}'.format(sim, num_sims - 1))
    this_vb_opt = simulate_and_fit(model, cond_init, true_params)
    assert(this_vb_opt.success == 1 or \
           this_vb_opt.message == 'A bad approximation caused failure to predict improvement.')
    #print(this_vb_opt.message)
    this_opt_x = model.objective.uncondition_x(this_vb_opt.x)
    #print('This opt x:', this_opt_x[0:2])
    
    opt_x_sims.append(copy.deepcopy(this_opt_x))
timer.toc('Simulations')
print('Done.')

Sim 0 of 199
Sim 20 of 199
Sim 40 of 199
Sim 60 of 199
Sim 80 of 199
Sim 100 of 199
Sim 120 of 199
Sim 140 of 199
Sim 160 of 199
Sim 180 of 199
Simulations: 628.6379897594452 seconds
Done.


In [7]:
print(this_vb_opt.message)
print(timer.time_dict)

Optimization terminated successfully.
{'Derivatives': 21.6134934425354, 'Simulations': 628.6379897594452}


In [8]:
# Write the result to a pickle file for use in R.

run_name = 'simulation'

pickle_output_filename = os.path.join(data_dir, '%s_python_refit_jackknife_results.pkl' % analysis_name)
pickle_output = open(pickle_output_filename, 'wb')

# Note that it does not seem that you can pickle a sparse Cholesky decomposition.
model.glmm_par.set_free(opt_x)
pickle_result_dict = logit_glmm.get_pickle_dictionary(model, kl_hess, moment_jac)
pickle_result_dict.update({
    'true_params': true_params,
    'glmm_par_sims': opt_x_sims,
    'weight_jac': obj_lib.pack_csr_matrix(weight_jac) 
  })

# Pickle dictionary using protocol 0.
pickle.dump(pickle_result_dict, pickle_output)
pickle_output.close()

print(pickle_output_filename)

print('\n\nDONE.')


/home/rgiordan/Documents/git_repos/LinearResponseVariationalBayes.py/Models/LogisticGLMM/data/simulated_data_for_refit_100_python_refit_jackknife_results.pkl


DONE.
